<a href="https://colab.research.google.com/github/obeabi/AirlineSentiment/blob/main/HighFlyersProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade yfinance
!pip install  --upgrade pandas_ta
!pip install ta pandas_ta
!pip install scipy==1.16.2

In [ ]:
import yfinance as yf
print(yf.__version__)
import pandas as pd
import numpy as np
from datetime import datetime
import time
import ta
import random
import requests
from datetime import datetime, timedelta
from scipy.stats import linregress
from transformers import pipeline
# ensure reproducibility
random.seed(42)
print("Libraries Installed!")

In [ ]:
today = datetime.today()
start_of_year = today.replace(month=1, day=1)

# First day of this month
first_day_month = today.replace(day=1)

# First day of this week (Monday as weekday 0)
first_day_week = today - timedelta(days=today.weekday())
print("First day of year:", start_of_year)

print("\nFirst day of this month:", first_day_month)
print("\nFirst day of this week:", first_day_week)

def most_recent_quarter_start(today=None):
    if today is None:
        today = pd.Timestamp.today().normalize()
    year = today.year
    month = today.month

    # Determine quarter start months: Jan, Apr, Jul, Oct
    if month >= 10:
        q_start = pd.Timestamp(year, 10, 1)
    elif month >= 7:
        q_start = pd.Timestamp(year, 7, 1)
    elif month >= 4:
        q_start = pd.Timestamp(year, 4, 1)
    else:
        q_start = pd.Timestamp(year, 1, 1)

    return q_start

# Example usage
print("Today:", pd.Timestamp.today().normalize())
print("Most recent quarter start:", most_recent_quarter_start())


In [ ]:
# List of ETFs to analyze
#df_o = pd.read_csv('stock_list.csv')

df_raw = pd.read_csv('small_list.csv')
df_raw = df_raw[df_raw['Type'].isin(['Stock'])]
recent_quarter = most_recent_quarter_start()
etfs = df_raw['Asset'].to_list()
print(etfs)

print(len(etfs))

## Filter for liquidity

In [ ]:

# Filter ETFs or stocks for liquidity
def filter_by_liquidity(etf_df, ticker_col="Asset", min_dollar_vol=10e6, lookback_days=30):
    liquid_etfs = []

    for ticker in etf_df[ticker_col]:
        try:
            # Fetch daily historical data
            data = yf.download(ticker, period=f"{lookback_days*2}d", interval="1d", auto_adjust=True)

            if data.empty:
                continue

            # Calculate dollar volume (Close × Volume)
            data["dollar_volume"] = data["Close"] * data["Volume"]

            # Calculate rolling average over lookback_days
            avg_dollar_volume = data["dollar_volume"].rolling(window=lookback_days).mean().iloc[-1]

            # Check liquidity condition
            if avg_dollar_volume >= min_dollar_vol:
                liquid_etfs.append(ticker)

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")

    # Return filtered DataFrame
    return etf_df[etf_df[ticker_col].isin(liquid_etfs)]

# Example usage
df = pd.DataFrame({"Assets": etfs})
liquid_df = filter_by_liquidity(df, ticker_col="Assets")
df_o = df_raw[df_raw['Asset'].isin(liquid_df['Assets'])]
etfs = df_o['Asset'].to_list()
print("")
print(etfs)
print(len(etfs))



# Classify Sector Stages

In [ ]:

def weinstein_stage(df, sma_window=30,smaSlope_window=10):
    """Determine Weinstein stage using 30-week SMA and its slope."""
     # Handle MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df["SMA"] = df["Close"].rolling(window=sma_window).mean()
    df["10_SMA"] = df["Close"].rolling(window=10).mean()

    # Compute linear regression slope on last N SMA points
    if len(df.dropna()) < sma_window:
        return None  # not enough data

    slope, _, _, _, _ = linregress(range(smaSlope_window), df["SMA"].tail(smaSlope_window))
    #slope_short, _, _, _, _ = linregress(range(smaSlope_window), df["10_SMA"].tail(smaSlope_window))

    latest_price = df["Close"].iloc[-1]
    latest_sma   = df["SMA"].iloc[-1]
    latest_10sma = df["10_SMA"].iloc[-1]

    # Determine stage
    if (latest_price > latest_sma) and (slope > 0) and (latest_price > latest_10sma) and (latest_10sma > latest_sma):
        stage = "Stage 2 (Advancing)"
    elif latest_price < latest_sma and slope < 0:
        stage = "Stage 4 (Declining)"
    elif np.abs(slope) <= 0.001:
        stage = "Stage 1 (Basing)"
    else:
        stage = "Stage 3 (Topping)"

    return stage, slope, latest_price, latest_sma, latest_10sma


In [ ]:

results = []
for etf in etfs:
    df = yf.download(etf, period="3y", interval="1wk", auto_adjust=True)
    stage_info = weinstein_stage(df)
    if stage_info:
        stage, slope, price, sma,sma_10 = stage_info
        results.append({
            "ETF": etf,
            "Stage": stage,
            "SMA_Slope": slope,
            "Latest_Price": price,
            "30W_SMA": sma,
            "10W_SMA": sma_10
        })

stages_df = pd.DataFrame(results).sort_values(by="SMA_Slope", ascending=False)
#print(len(stages_df))
stages_df.head()

In [ ]:
advancing_stocks= stages_df[stages_df["Stage"] .isin(["Stage 2 (Advancing)"]) ]
advancing_stocks.reset_index(drop=True, inplace=True)
advancing_stocks.head()

In [ ]:
# List of ETFs to analyze
df_o = df_o[df_o['Asset'].isin(advancing_stocks['ETF'])]
#df_raw = pd.read_csv('etf_list.csv')
#df_raw = df_raw[df_raw['Type'].isin(['ETF','Stock'])]
#recent_quarter = most_recent_quarter_start()
etfs = df_o['Asset'].to_list()
print(etfs)

print(len(etfs))

In [ ]:

def macdv(prices, fast=12, slow=26, signal=9, atr_window=10, thresholds=(50, 150)):
    """
    Compute MACD-V (volatility normalized MACD).

    Parameters
    ----------
    prices : pd.Series
        Price series (e.g. closing prices).
    fast : int
        Fast EMA period.
    slow : int
        Slow EMA period.
    signal : int
        Signal EMA period for MACD line.
    atr_window : int
        ATR lookback window.
    thresholds : tuple
        (lower, upper) thresholds for neutral/ranging and extreme momentum zones.

    Returns
    -------
    pd.DataFrame with columns:
        - MACDV : MACD-V value
        - Signal : EMA of MACDV
        - Histogram : MACDV - Signal
        - EntryFlag : True when momentum is strong enough, False otherwise
    """
    # --- Step 1: EMAs for MACD ---
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd_raw = ema_fast - ema_slow

    # --- Step 2: ATR for normalization ---
    high = prices.shift(1) * (1 + 0.01)   # synthetic highs/lows if OHLC not available
    low = prices.shift(1) * (1 - 0.01)
    close = prices
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(atr_window).mean()

    # --- Step 3: Normalize MACD by ATR ---
    macdv = (macd_raw / atr) * 100

    # --- Step 4: Signal line and histogram ---
    signal_line = macdv.ewm(span=signal, adjust=False).mean()
    histogram = macdv - signal_line

    # --- Step 5: Entry conditions ---
    lower, upper = thresholds
    entry_flag = ((macdv.abs() > lower) & (macdv.abs() < upper)) | (macdv.abs() > upper)

    df = pd.DataFrame({
        "MACDV": macdv,
        "Signal": signal_line,
        "Histogram": histogram,
        "EntryFlag": entry_flag
    })
    curr = df.iloc[-1]

    return curr.EntryFlag

def money_flow_signals(df, period=10):
    """
    Calculate Money Flow Index (MFI) and generate signals:
    - Positive money flow (TP > TP_prev)
    - Divergence (Price vs MFI mismatch)

    Parameters:
        df (pd.DataFrame): DataFrame with columns ["High", "Low", "Close", "Volume"]
        period (int): Lookback period for MFI (default=10)

    Returns:
        pd.DataFrame with added columns: ["TypicalPrice", "MFI", "PositiveFlow", "Divergence"]
    """

    df = df.copy()

    # Step 1: Typical Price
    df["TypicalPrice"] = (df["High"] + df["Low"] + df["Close"]) / 3

    # Step 2: Raw Money Flow
    df["RawMoneyFlow"] = df["TypicalPrice"] * df["Volume"]

    # Step 3: Positive & Negative Flow
    df["PositiveFlow"] = np.where(df["TypicalPrice"] > df["TypicalPrice"].shift(1), df["RawMoneyFlow"], 0.0)
    df["NegativeFlow"] = np.where(df["TypicalPrice"] < df["TypicalPrice"].shift(1), df["RawMoneyFlow"], 0.0)

    # Step 4: Money Flow Ratio & MFI
    pos_flow = df["PositiveFlow"].rolling(period).sum()
    neg_flow = df["NegativeFlow"].rolling(period).sum()
    money_flow_ratio = pos_flow / neg_flow.replace(0, np.nan)
    df["MFI"] = 100 - (100 / (1 + money_flow_ratio))

    # Step 5: Positive Money Flow Signal
    df["PositiveFlowSignal"] = df["TypicalPrice"] > df["TypicalPrice"].shift(1)

    # Step 6: Divergence Detection
    df["PriceHigh"] = df["Close"].rolling(period).max()
    df["PriceLow"] = df["Close"].rolling(period).min()
    df["MFIHigh"] = df["MFI"].rolling(period).max()
    df["MFILow"] = df["MFI"].rolling(period).min()

    def divergence(row):
        if np.isnan(row["MFI"]):
            return None
        # Price makes higher high, but MFI does not
        if row["Close"] >= row["PriceHigh"] and row["MFI"] < row["MFIHigh"]:
            return "Bearish Divergence ⚠️"
        # Price makes lower low, but MFI does not
        elif row["Close"] <= row["PriceLow"] and row["MFI"] > row["MFILow"]:
            return "Bullish Divergence ✅"
        else:
            return "No Divergence"

    df["Divergence"] = df.apply(divergence, axis=1)
    df['Entry_signal']= (df["MFI"] > 50) & (df["Divergence"] == "No Divergence")
    curr = df.iloc[-1]

    return curr.Entry_signal




In [ ]:
# Function to fetch historical weekly data
def rolling_regression_slope(series, window=10):
    """Rolling linear regression slope (price units per bar)."""
    def calc_slope(y):
        if len(y) < 2:
            return np.nan
        x = np.arange(len(y))
        slope, _, _, _, _ = linregress(x, y)
        return slope
    return series.rolling(window).apply(calc_slope, raw=False)

def anchored_vwap(ticker: str, lookback_weeks: int = 5):
    """
    Calculate the Anchored VWAP from the most recent high within the past `lookback_weeks`
    and create a 'Signal' column that gives 'Buy' when Close > Anchored_VWAP, else 'No-Buy'.

    Args:
        ticker (str): Stock ticker symbol
        lookback_weeks (int): Number of weeks to look back for the highest price

    Returns:
        pandas.DataFrame: DataFrame with OHLC, Volume, Anchored_VWAP, and Signal columns
    """
    try:
        # --- Fetch 6 months of daily data to cover the lookback window
        data = yf.download(ticker, period="6mo", interval="1d", progress=False,auto_adjust=True)
        if isinstance(data.columns, pd.MultiIndex):
          data.columns = data.columns.get_level_values(0)  # keep only first level

        if data.empty:
            raise ValueError(f"No data retrieved for ticker {ticker}")

        data.dropna(inplace=True)
        data.index = pd.to_datetime(data.index)

        # --- Ensure we have enough data for the lookback period
        min_days_required = lookback_weeks * 10  # ~5 trading days per week
        if len(data) < min_days_required:
            raise ValueError(f"Insufficient data: {len(data)} days available, {min_days_required} required")

        # --- Find the most recent high within the past lookback_weeks
        recent_period = data.tail(min_days_required)
        anchor_date = recent_period['High'].idxmax()

        # --- Verify anchor_date is a valid Timestamp
        if not isinstance(anchor_date, pd.Timestamp):
            raise ValueError(f"Invalid anchor_date: {anchor_date}. Expected a Timestamp.")

        anchor_price = recent_period.loc[anchor_date, 'High']

        # --- Use .date() safely since we confirmed anchor_date is a Timestamp
        print(f"\nAnchored VWAP for {ticker} starting from {anchor_date.date()} (recent high = {anchor_price:.2f})")

        # --- Slice data from the anchor date onwards
        anchor_data = data.loc[anchor_date:]

        # --- Compute VWAP starting from the anchor date
        # Use typical price ((H+L+C)/3) for more accurate VWAP
        typical_price = (anchor_data['High'] + anchor_data['Low'] + anchor_data['Close']) / 3
        q = anchor_data['Volume']
        pv = (typical_price * q).cumsum()
        v = q.cumsum()

        # --- Avoid division by zero
        avwap = pv / v.where(v != 0, np.nan)

        # --- Add Anchored VWAP to the full dataset
        data['Anchored_VWAP'] = np.nan  # Initialize with NaN
        data.loc[anchor_date:, 'Anchored_VWAP'] = avwap
        # --- Add swing high information to the DataFrame
        data['Swing_High_Price'] = anchor_price
        data['Swing_High_Date'] = anchor_date

        # --- Create Buy/No-Buy Signal
        # Only apply signal where Anchored_VWAP is not NaN
        data['Signal'] = np.where(
            (data['Close'] > data['Anchored_VWAP']) & (data['Anchored_VWAP'].notna()),
            True,
            False
        )

        return data[['Anchored_VWAP', 'Signal','Swing_High_Price']]

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


def anchored_vwap_old(ticker, anchor_date):
    """
    Calculate Anchored VWAP starting from a given anchor_date.
    Works with both single-level and multi-level columns (e.g. yfinance output).
    """
    # --- Step 1: Flatten columns if multi-index ---
    df = yf.download(ticker, period="1y", interval="1d",auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)  # keep only first level

    # --- Step 2: Ensure required columns exist ---
    required_cols = ["High", "Low", "Close", "Volume"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # --- Step 3: Subset from anchor_date ---
    df_anchor = df.loc[df.index >= pd.to_datetime(anchor_date)].copy()
    if df_anchor.empty:
        raise ValueError(f"No data found on/after {anchor_date}")

    # --- Step 4: Compute typical price ---
    df_anchor["typical_price"] = (df_anchor["High"] + df_anchor["Low"] + df_anchor["Close"]) / 3.0

    # --- Step 5: Cumulative PV and VWAP ---
    df_anchor["cum_pv"] = (df_anchor["typical_price"].astype(float) * df_anchor["Volume"].astype(float)).cumsum()
    df_anchor["cum_vol"] = df_anchor["Volume"].astype(float).cumsum()
    df_anchor["anchored_vwap"] = df_anchor["cum_pv"] / df_anchor["cum_vol"]

    return df_anchor[["anchored_vwap"]]

def get_monthly_data(ticker):
  try:
      df = yf.download(ticker, period="10y", interval="1mo",auto_adjust=True)

      #if isinstance(df.columns, pd.MultiIndex):
        #df.columns = df.columns.get_level_values(0)  # keep only first level
      # Compute MACD using ta
      df['10_month_SMA'] = df['Close'].rolling(window=10).mean()
      df['slope_raw'] = rolling_regression_slope(df['10_month_SMA'], window=5)
      # Calculate MACD and Signal Line
      df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
      df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
      # Compute MACD Line
      df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
      # Compute Signal Line (9-day EMA of MACD Line)
      df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
      df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
      #df['AvgVolume'] = df["Volume"].rolling(window=10).mean()
      df['RVOL'] = df['Volume'] / df['Volume'].rolling(window=10).mean()
      df['RVOL_Slope'] = df['RVOL'].diff()
      # Calculate ADX, +DMI and -DMI
      high = df['High']
      low = df['Low']
      close = df['Close']
      # Calculate directional movements
      up_move = high.diff()
      down_move = -low.diff()
      plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
      minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
      # Calculate True Range (TR)
      tr1 = high - low
      tr2 = (high - close.shift()).abs()
      tr3 = (low - close.shift()).abs()
      tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
      # Smooth TR, +DM, and -DM using Wilder’s smoothing
      atr = tr.rolling(window=14).sum()
      plus_dm_series = pd.Series(plus_dm.ravel(), index=df.index).astype(float)
      minus_dm_series = pd.Series(minus_dm.ravel(), index=df.index).astype(float)
      plus_dm_smoothed = plus_dm_series.rolling(window=14).sum()
      minus_dm_smoothed = minus_dm_series.rolling(window=14).sum()
      # Directional Indicators
      plus_di = 100 * (plus_dm_smoothed / atr)
      minus_di = 100 * (minus_dm_smoothed / atr)
      # DX and ADX
      dx = 100 * (np.abs(plus_di - minus_di) / (plus_di + minus_di))
      adx = dx.rolling(window=14).mean()
      # Add results to original DataFrame
      df['+DI'] = plus_di
      df['-DI'] = minus_di
      df['ADX'] = adx
      df['di_flag'] = df['+DI'] > df['-DI']
      df['di_flag'] = df['di_flag'].astype(int)
      df['adx_indicator'] = np.where(df['ADX'] > 10, 1, 0)
      df['adx_signal'] = df['adx_indicator'] * df['di_flag']

      return df
  except Exception as e:
      print("There is an error getting monthly data", e)

def get_weekly_data(ticker):
  try:
      df = yf.download(ticker, period="10y", interval="1wk",auto_adjust=True)
      #if isinstance(df.columns, pd.MultiIndex):
        #df.columns = df.columns.get_level_values(0)  # keep only first level
      df['10_week_SMA'] = df['Close'].rolling(window=10).mean()
      df['30_week_SMA'] = df['Close'].rolling(window=30).mean()
      df['slope_raw'] = rolling_regression_slope(df['30_week_SMA'], window=10)
      df['slope_pct_per_week'] = df['slope_raw'] / df['30_week_SMA']          # fractional change per week
      df['SMA_Slope'] = df['slope_pct_per_week'] * 52 * 100       # % per year
      # Optional: also keep a simple angle if you still want it
      df['slope_angle_deg'] = np.degrees(np.arctan(df['slope_pct_per_week'] * 52))
      df['10_EMA'] = df['Close'].ewm(span=10, adjust=False).mean()
      df['20_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()
      df['ATR'] = compute_atr(df, 14)
      df['OBV'] = compute_obv(df)
      obv_slope, _, _, _, _ = linregress(range(30), df["OBV"].tail(30))
      df['OBV_Slope'] = obv_slope
      df['30_week_avg_volume'] = df['Volume'].rolling(window=30).mean()
      # Calculate MACD and Signal Line
      df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
      df['21_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
      df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
      # Compute MACD Line
      df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
      # Compute Signal Line (9-day EMA of MACD Line)
      df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
      df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
      # Compute raw EFI
      df['EFI'] = (df['Close'].diff()) * df['Volume']
      # Compute EMA of EFI
      df['EFI_EMA'] = df['EFI'].ewm(span=13, adjust=False).mean()
      # Determine if EFI_EMA is rising or falling
      df['EFI_EMA_Trend'] = df['EFI_EMA'].diff().apply(lambda x: 'Rising' if x > 0 else 'Falling')
      # Calculate ADX, +DMI and -DMI
      high = df['High']
      low = df['Low']
      close = df['Close']
      # Calculate directional movements
      up_move = high.diff()
      down_move = -low.diff()
      plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
      minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
      # Calculate True Range (TR)
      tr1 = high - low
      tr2 = (high - close.shift()).abs()
      tr3 = (low - close.shift()).abs()
      tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
      # Smooth TR, +DM, and -DM using Wilder’s smoothing
      atr = tr.rolling(window=14).sum()
      plus_dm_series = pd.Series(plus_dm.ravel(), index=df.index).astype(float)
      minus_dm_series = pd.Series(minus_dm.ravel(), index=df.index).astype(float)
      plus_dm_smoothed = plus_dm_series.rolling(window=14).sum()
      minus_dm_smoothed = minus_dm_series.rolling(window=14).sum()
      # Directional Indicators
      plus_di = 100 * (plus_dm_smoothed / atr)
      minus_di = 100 * (minus_dm_smoothed / atr)
      # DX and ADX
      dx = 100 * (np.abs(plus_di - minus_di) / (plus_di + minus_di))
      adx = dx.rolling(window=14).mean()
      # Add results to original DataFrame
      df['+DI'] = plus_di
      df['-DI'] = minus_di
      df['ADX'] = adx
      df['di_flag'] = df['+DI'] > df['-DI']
      df['di_flag'] = df['di_flag'].astype(int)
      df['adx_indicator'] = np.where(df['ADX'] > 10, 1, 0)
      df['adx_signal'] = df['adx_indicator'] * df['di_flag']

      # --- Overhead Resistance Filter ---
      recent_52_weeks = df[-52:]
      max_close_52w = recent_52_weeks['Close'].max().iloc[0]
      max_price = df['Close'].max().iloc[0]
      last_close = df['Close'].iloc[-1].iloc[0]
      # Filter condition: Close is within 15% of 52-week high
      df['No_Overhead_Resistance'] = last_close > (max_close_52w*0.80)
      # --- Above 52 weeks High ---
      df['above_52w_high'] = last_close > max_close_52w
      df['below_52w_high'] = last_close < max_close_52w

      return df
  except Exception as e:
      print("There is an error getting weekly data", e)

def is_macd_bullish(df):
    """
    Determines if there is a bullish signal on the MACD indicator.
    A bullish signal occurs when the MACD line is above the signal line.

    Parameters:
    - df: dataframe. Must have columns 'MACD_Line' and 'Signal_Line'.
    Returns:
    - bool: True if a bullish signal is detected, otherwise False.
    """
    try:
      if df.empty:
        return False
      # Calculate MACD histogram if not already present

      curr = df.iloc[-3:]
      macd_crossover = curr['MACD_Line'].iloc[-1] > curr['Signal_Line'].iloc[-1]
      hist_increasing = (curr['MACD_Hist'].iloc[-1] > curr['MACD_Hist'].iloc[-2]) \
                         or (curr['MACD_Hist'].iloc[-2] > curr['MACD_Hist'].iloc[-3])

      return macd_crossover
              #and (hist_increasing or curr['MACD_Hist'].iloc[-1] > 0)
    except Exception as e:
      print("Something went wrong while computing the MACD", e)

def is_macd_bullish_hr(df):
    """
    Determines if there is a bullish signal on the MACD indicator on hourly chart.
    A bullish signal occurs when the MACD line is above the signal line.

    Parameters:
    - df: dataframe. Must have columns 'MACD_Line' and 'Signal_Line'.
    Returns:
    - bool: True if a bullish signal is detected, otherwise False.
    """
    try:
      if df.empty:
        return False
      # Calculate MACD histogram if not already present

      curr = df.iloc[-3:]
      macd_crossover = curr['MACD_Line'].iloc[-1] > curr['Signal_Line'].iloc[-1]

      return macd_crossover
             # and curr['MACD_Hist'].iloc[-1] > 0
    except Exception as e:
      print("Something went wrong while computing the MACD", e)

def is_macd_bullish_min(df):
    """
    Determines if there is a bullish signal on the MACD indicator on 15 minute chart.
    A bullish signal occurs when the MACD line is above the signal line.

    Parameters:
    - df: dataframe. Must have columns 'MACD_Line' and 'Signal_Line'.
    Returns:
    - bool: True if a bullish signal is detected, otherwise False.
    """
    try:
      if df.empty:
        return False
      # Calculate MACD histogram if not already present

      curr = df.iloc[-3:]
      macd_crossover = curr['MACD_Line'].iloc[-1] > curr['Signal_Line'].iloc[-1]

      return macd_crossover
    except Exception as e:
      print("Something went wrong while computing the MACD", e)

def calculate_bbw(df, window=20):
    """ Calculates Bollinger Bands Width"""
    sma = df['Close'].rolling(window).mean()
    std = df['Close'].rolling(window).std()
    upper = sma + 2*std
    lower = sma - 2*std
    bbw = (upper - lower) / sma
    return bbw

def calculate_vwap(df):
  """ Calculates vwap"""
  try:
    Typical_Price = (df['Close'].values + df['High'].values + df['Low'].values) / 3
    TPV = Typical_Price * df['Volume'].values
    vwap = TPV.cumsum() / df['Volume'].values.cumsum()
    return vwap
  except Exception as e:
    print("Something went wrong while computing the VWAP:", e)
    return None

# Function to compute RSI
def compute_rsi(series, period=10):
  try:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
  except Exception as e:
        print("Something went wrong while computing the RSI:", e)
        return None

def calculate_ma(data, length=10, ma_type="WMA"):
    if ma_type == "SMA":
        return data.rolling(window=length).mean()
    elif ma_type == "EMA":
        return data.ewm(span=length, adjust=False).mean()
    elif ma_type == "WMA":
        weights = np.arange(1, length+1)
        return data.rolling(length).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
    elif ma_type == "VWMA":
        return ta.volume_weighted_average_price(data, length)

def calculate_bollinger_bands(data, length=10, std_dev=2.0):
    sma = data.rolling(window=length).mean()
    std_dev = data.rolling(window=length).std()
    upper_band = sma + (std_dev * std_dev)
    lower_band = sma - (std_dev * std_dev)
    return upper_band, lower_band

# Function to compute ATR (Average True Range)
def compute_atr(df, period=10):
  try:
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df['TR'].rolling(window=period).mean()
  except Exception as e:
      print("Something went wrong whilecomputing the ATR", e)


# Function to compute On-Balance Volume (OBV)
def compute_obv(df):
  try:
    # Calculate daily price change: 1 if price is up, -1 if down, 0 if unchanged
    price_change = df['Close'].diff()

    # Use price change to decide whether to add or subtract volume
    obv = (price_change > 0).astype(int) * df['Volume']  # Volume when price goes up
    obv -= (price_change < 0).astype(int) * df['Volume']  # Volume when price goes down

    # We accumulate the OBV by taking the cumulative sum of the volume changes
    obv = obv.cumsum()

    return obv
  except Exception as e:
      print("Something went wrong while computing the OBV", e)

def is_bullish_engulfing(df):
    prev = df.iloc[-2]
    curr = df.iloc[-1]
    return (
        prev['Close'].iloc[0] < prev['Open'].iloc[0] and # Previous red
        curr['Close'].iloc[0] > curr['Open'].iloc[0] and # Current green
        curr['Close'].iloc[0] > prev['Open'].iloc[0] and
        curr['Open'].iloc[0] < prev['Close'].iloc[0]
    )

# Function to calculate risk-reward ratio
def calculate_risk_reward(df):
  try:
    if df.empty or len(df) < 20:  # Ensure there are enough data points
        return np.nan

    #latest_price = df['Close'].iloc[-1].iloc[0]
    latest_price = df['Close'].iloc[-1]

    # Use the ATR for setting support level
    atr = df['ATR'].iloc[-1]  # Latest ATR value
    price_ema = df['8_day_EMA'].iloc[-1]
    atr_multiple = 2.25  # You can adjust this multiplier based on your strategy

    # Calculate the support level using the ATR
    trailing = atr
    support_level = price_ema - (trailing * atr_multiple)
    resistance_level = price_ema + (trailing * atr_multiple)

    return support_level, latest_price, trailing,resistance_level
  except Exception as e:
      print("Something went wrong while computing the reward-risk ratio", e)

# Function to fetch daily data
def get_daily_data(ticker):
    df = yf.download(ticker, period="1y", interval="1d",auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)  # keep only first level
    df['20_day_SMA'] = df['Close'].rolling(window=20).mean()
    df['20_day_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['50_day_avg_volume'] = df['Volume'].rolling(window=50).mean()
    df['8_day_EMA'] = df['Close'].ewm(span=8, adjust=False).mean()
    df['15_day_EMA'] = df['Close'].ewm(span=15, adjust=False).mean()
    df['21_day_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['26_day_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['5_day_SMA'] = df['Close'].rolling(window=5).mean()
    df['50_day_SMA'] = df['Close'].rolling(window=50).mean()
    df['100_day_SMA'] = df['Close'].rolling(window=100).mean()
    df['200_day_SMA'] = df['Close'].rolling(window=200).mean()
    # Slope for 5-day SMA (short-term trend)
    df['slope5_raw'] = rolling_regression_slope(df['5_day_SMA'], window=10)
    df['slope5_pct_per_day'] = df['slope5_raw'] / df['5_day_SMA']  # fractional change per day
    df['slope5_annualized_pct'] = df['slope5_pct_per_day'] * 252 * 100  # % per year
    # Slope for 50-day SMA (intermediate-term trend)
    df['slope50_raw'] = rolling_regression_slope(df['50_day_SMA'], window=10)
    df['slope50_pct_per_day'] = df['slope50_raw'] / df['50_day_SMA']  # fractional change per day
    df['slope50_annualized_pct'] = df['slope50_pct_per_day'] * 252 * 100  # % per year
    df['ATR'] = compute_atr(df, 10)
    df["8EMA_plus_ATR"] = df["8_day_EMA"] + 0.5* df["ATR"]
    df["8EMA_plus_ATRL"] = df["8_day_EMA"] + 1* df["ATR"]
    # Compute MACD using ta
    df["MACD_Line"] = ta.trend.macd(df["Close"], window_slow=26, window_fast=12)
    df["Signal_Line"] = ta.trend.macd_signal(df["Close"], window_slow=26, window_fast=12, window_sign=9)
    df["MACD_Hist"] = ta.trend.macd_diff(df["Close"], window_slow=26, window_fast=12, window_sign=9)
    df["MACD_Hist_above_zero"] = df["MACD_Hist"] > 0
    df["MACD_Hist_below_zero"] = df["MACD_Hist"] < 0
    df['macd_above_signal'] = df['MACD_Line'] > df['Signal_Line']
    df['macd_below_signal'] = df['MACD_Line'] < df['Signal_Line']
    df['VWAP'] = calculate_vwap(df)
    # Compute raw EFI
    df['EFI'] = (df['Close'].diff()) * df['Volume']
    # Compute EMA of EFI
    df['EFI_EMA'] = df['EFI'].ewm(span=13, adjust=False).mean()
    # Determine if EFI_EMA is rising or falling
    df['EFI_EMA_Trend'] = df['EFI_EMA'].diff().apply(lambda x: 'Rising' if x > 0 else 'Falling')
    # Calculate ADX, +DMI and -DMI
    high = df['High']
    low = df['Low']
    close = df['Close']
    # Calculate directional movements
    up_move = high.diff()
    down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    # Calculate True Range (TR)
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    # Smooth TR, +DM, and -DM using Wilder’s smoothing
    atr = tr.rolling(window=14).sum()
    plus_dm_series = pd.Series(plus_dm.ravel(), index=df.index).astype(float)
    minus_dm_series = pd.Series(minus_dm.ravel(), index=df.index).astype(float)
    plus_dm_smoothed = plus_dm_series.rolling(window=14).sum()
    minus_dm_smoothed = minus_dm_series.rolling(window=14).sum()
    # Directional Indicators
    plus_di = 100 * (plus_dm_smoothed / atr)
    minus_di = 100 * (minus_dm_smoothed / atr)
    # DX and ADX
    dx = 100 * (np.abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window=14).mean()
    # Add results to original DataFrame
    df['+DI'] = plus_di
    df['-DI'] = minus_di
    df['ADX'] = adx
    df['di_flag'] = df['+DI'] > df['-DI']
    df['di_flag'] = df['di_flag'].astype(int)
    df['adx_indicator'] = np.where(df['ADX'] > 14, 1, 0)
    df['adx_signal'] = df['adx_indicator'] * df['di_flag']

    return df

# Function to fetch hourly data
def get_30min_data(ticker):
    df = yf.download(ticker, interval='30m', period='60d',auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.get_level_values(0)  # keep only first level

    df['65d_SMA'] = df['Close'].rolling(window=65).mean()
    # Slope calculation: regression over recent 10 bars (~5 trading hours)
    df['slope_raw'] = rolling_regression_slope(df['65d_SMA'], window=10)
    # Normalize → fractional change per 30-minute bar
    df['slope_norm'] = df['slope_raw'] / df['65d_SMA']
    # Annualize to % per year (standard for 30m regular hours)
    bars_per_year = 252 * 13
    df['SMA_Slope'] = df['slope_norm'] * bars_per_year * 100
    # Short-term & medium-term moving averages
    df['21_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['50_EMA'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['200_EMA'] = df['Close'].ewm(span=200, adjust=False).mean()
    # Calculate MACD and Signal Line
    df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    # Compute MACD Line
    df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
    # Compute Signal Line (9-day EMA of MACD Line)
    df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
    return df


# Function to fetch hourly data
def get_15min_data(ticker):
    df = yf.download(ticker, interval='15m', period='30d',auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.get_level_values(0)  # keep only first level

    df['130d_SMA'] = df['Close'].rolling(window=130).mean()
    # Slope calculation: regression over recent 10 bars (~5 trading hours)
    df['slope_raw'] = rolling_regression_slope(df['130d_SMA'], window=10)
    # Normalize → fractional change per 15-minute bar
    df['slope_norm'] = df['slope_raw'] / df['130d_SMA']
    # Annualize to % per year (standard for 30m regular hours)
    bars_per_year = 252 * 26
    df['SMA_Slope'] = df['slope_norm'] * bars_per_year * 100
    # Short-term & medium-term moving averages
    df['21_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['50_EMA'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['200_EMA'] = df['Close'].ewm(span=200, adjust=False).mean()
    return df


# Function to check monthly trend
def is_monthly_trend_bullish(df):
    if df.empty:
        return False


    adx_ok              = df['adx_signal'].iloc[-1] == 1
    latest_price        = df['Close'].iloc[-1].iloc[0]
    latest_sma          = df['10_month_SMA'].iloc[-1]
    macd_bullish_signal = is_macd_bullish(df)
    above_10_month_SMA  = (latest_price > latest_sma)

    return above_10_month_SMA and macd_bullish_signal and adx_ok


# Function to check weekly trend
def is_weekly_trend_bullish(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    latest_30sma = df['30_week_SMA'].iloc[-1]
    latest_10sma = df['10_week_SMA'].iloc[-1]
    sma_10_above_30 = latest_10sma > latest_30sma
    above_10_week_SMA = latest_price > latest_10sma
    above_30_week_SMA = latest_price > latest_30sma
    sma_slope = df['SMA_Slope'].iloc[-1]> 30
    obv_slope = df['OBV_Slope'].iloc[-1]> 0
    macd_bullish_signal =  is_macd_bullish(df)
    elderforce_trend_ok = df['EFI_EMA_Trend'].iloc[-1] == 'Rising'
    adx_ok = df['adx_signal'].iloc[-1] == 1
    elderforce_ema_ok = df['EFI_EMA'].iloc[-1] > 0
    volume_ok = df['Volume'].iloc[-1] > df['30_week_avg_volume'].iloc[-1] # Institutional interest
    volume_ok = volume_ok.iloc[0]
    # Calculate the OBV Moving Average
    df['OBV_EMA'] = df['OBV'].ewm(span=10, adjust=False).mean()
    # OBV trending up if current OBV is above the 30-period EMA
    obv_trending_up = df['OBV'].iloc[-1] > df['OBV_EMA'].iloc[-1]
    # OBV trending down if current OBV is below the 30-period EMA
    obv_trending_down = df['OBV'].iloc[-1] < df['OBV_EMA'].iloc[-1]
    trend_ok = sma_10_above_30  and above_10_week_SMA and above_30_week_SMA and adx_ok
    no_overhead_supply = df['No_Overhead_Resistance'].iloc[-1]
    above_52w_high = df['above_52w_high'].iloc[-1]
    time.sleep(2)  # Add a delay of 1 second between requests

    return  trend_ok and sma_slope # and macd_bullish_signal



# Function to check daily entry signal
def is_daily_entry_signal(df2):
    if df2.empty:
        return False

    df = df2.copy()
    latest_price = df['Close'].iloc[-1]
    sma_slope_5  = df['slope5_annualized_pct'].iloc[-1] > 50
    sma_slope_50 = df['slope50_annualized_pct'].iloc[-1] > 50
    latest_8ema = df['8_day_EMA'].iloc[-1]
    latest_5sma = df['5_day_SMA'].iloc[-1]
    latest_15ema = df['15_day_EMA'].iloc[-1]
    latest_20sma = df['20_day_SMA'].iloc[-1]
    latest_50sma = df['50_day_SMA'].iloc[-1]
    latest_100sma = df['100_day_SMA'].iloc[-1]
    latest_200sma = df['200_day_SMA'].iloc[-1]
    above_20sma = latest_price > latest_20sma
    above_50sma = latest_price > latest_50sma
    above_100sma = latest_price > latest_100sma
    above_200sma = latest_price > latest_200sma
    above_8ema = latest_price >= latest_5sma
    is_8ema_above_15ema = latest_8ema > latest_15ema
    is_20sma_above_50sma = latest_20sma > latest_50sma
    is_50sma_above_100sma = latest_50sma > latest_100sma
    is_50sma_above_200sma = latest_50sma > latest_200sma
    is_100sma_above_200sma = latest_100sma > latest_200sma
    volume_ok = df['Volume'].iloc[-1] >= 1.1* df['50_day_avg_volume'].iloc[-1] # Institutional interest
    volume_ok = volume_ok
    macd_bullish_signal = df["MACD_Hist_above_zero"].iloc[-1] #is_macd_bullish(df)
    #vwap_price = df['VWAP'].iloc[-1]
    elderforce_trend_ok = df['EFI_EMA_Trend'].iloc[-1] == 'Rising'
    elderforce_ema_ok = df['EFI_EMA'].iloc[-1] > 0
    adx_ok = df['adx_signal'].iloc[-1] == 1
    slopes_ok =  sma_slope_5 and sma_slope_50
    moving_averages_ok = above_50sma and above_100sma and above_200sma \
                          and is_50sma_above_100sma \
                          and is_50sma_above_200sma and is_100sma_above_200sma \

    # Look for a breakout above 20-day SMA & RSI > 50
    return moving_averages_ok and slopes_ok and adx_ok


def get_heikin_ashi_signal(ticker="AAPL", period="6mo", interval="1d"):
    # Fetch OHLC data
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)  # keep only first level

    # Compute Heikin Ashi candles
    ha_df = pd.DataFrame(index=df.index)
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4

    ha_open = []
    for i in range(len(df)):
        if i == 0:
             ha_open.append((df['Open'].iloc[i] + df['Close'].iloc[i]) / 2)
        else:
            ha_open.append((ha_open[i-1] + ha_df['HA_Close'].iloc[i-1]) / 2)
    ha_df['HA_Open'] = ha_open
    ha_df['HA_High'] = ha_df[['HA_Open', 'HA_Close']].assign(High=df['High']).max(axis=1)
    ha_df['HA_Low'] = ha_df[['HA_Open', 'HA_Close']].assign(Low=df['Low']).min(axis=1)

    # Combine with original
    df = df.join(ha_df)
    # Check for green candle with flat bottom
    last = df.iloc[-1]
    green_candle = last['HA_Close'] > last['HA_Open']
    flat_bottom = abs(last['HA_Open'] - last['HA_Low']) < 0.01  # tiny wick or flat bottom tolerance
    signal = green_candle and flat_bottom

    print(f"\n🔍 Checking {ticker} ({interval} timeframe)")
    print(f"HA_Open: {last['HA_Open']:.2f}, HA_Close: {last['HA_Close']:.2f}, HA_Low: {last['HA_Low']:.2f}")
    if signal:
        print("✅ Heikin Ashi candle is GREEN with a FLAT bottom — strong bullish signal!")
    elif green_candle:
        print("🟢 Candle is green but not flat-bottomed — still bullish, but less strong.")
    else:
        print("🔴 Not a bullish candle — no entry confirmation yet.")

    return signal, green_candle

# Check entry conditions
def check_entry_conditions(tickers):
    results = []
    for ticker in tickers:
      df = get_daily_data(ticker)
      latest_price = df['Close'].iloc[-1]
      prev_price = df['Close'].iloc[-2]
      latest_sma = df['50_day_SMA'].iloc[-1]
      latest_price_8ema =df['8_day_EMA'].iloc[-1]
      latest_price_21ema =df['21_day_EMA'].iloc[-1]
      price_threshold_ATR = df['8EMA_plus_ATR'].iloc[-1]
      price_threshold_ATRL = df['8EMA_plus_ATRL'].iloc[-1]
      macd_bullish_signal = df['macd_above_signal'].iloc[-1]
      mfi_signal = money_flow_signals(df)
      # Print results
      print(f"\nMoney flow indicator for {ticker} is:")
      print(mfi_signal)
      macdv_signal = macdv(df['Close'])
      print(f"\nMacd-V indicator for {ticker} is:")
      print(macdv_signal )


      df_entry              = get_30min_data(ticker)
      latest_priceh_5sma    = df_entry['65d_SMA'].iloc[-1]
      latest_priceh         = df_entry['Close'].iloc[-1]
      sma_slope_h           = df_entry['SMA_Slope'].iloc[-1]> 30
      HA_buy_signal_h,gc_h  = get_heikin_ashi_signal(ticker, period="90d", interval="1d")

      df_refined_entry      = get_15min_data(ticker)
      latest_pricem_5sma    = df_refined_entry['130d_SMA'].iloc[-1]
      latest_pricem         = df_refined_entry['Close'].iloc[-1]
      sma_slope_m           = df_refined_entry['SMA_Slope'].iloc[-1]> 30


      refined_entry_signal =  sma_slope_h or sma_slope_m


      if latest_price > price_threshold_ATRL:
        entry_signal = "Extended Momentum Entry"
      elif latest_price >= latest_price_8ema and (latest_price <= price_threshold_ATR) :
        entry_signal = "Aline Entry"
      elif (latest_price > price_threshold_ATR) and (latest_price <= price_threshold_ATRL) and refined_entry_signal  :
        entry_signal = "True Trend Entry"
      elif (latest_price < latest_price_8ema) and (latest_price >= latest_price_21ema) :
          entry_signal= "Bline Entry"
      elif  (latest_price <= latest_price_21ema) and (latest_price >= latest_sma) :
          entry_signal = "Below Bline Entry"
      elif  latest_price < latest_sma:
          entry_signal = "Bearish"

      else:
        entry_signal = "Other"
      results.append([ticker, entry_signal])
    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["Asset", "Entry_Signal"])
    return df_results

# Multi-timeframe strategy check returning a DataFrame
def check_mtf_entry(tickers):
    results = []

    for ticker in tickers:
        monthly_df = get_monthly_data(ticker)
        weekly_df = get_weekly_data(ticker)
        daily_df = get_daily_data(ticker)

        if is_weekly_trend_bullish(weekly_df) and is_monthly_trend_bullish(monthly_df):
            if is_daily_entry_signal(daily_df):
                entry_signal = "Entry Confirmed ✅"
                results.append([ticker, entry_signal])
            else:
                entry_signal = "No Entry Yet on Daily Timeframe ⏳"
                #results.append([ticker, entry_signal])
        else:
            entry_signal = "Monthly or Weekly Trend Not Bullish ❌"
            #results.append([ticker, entry_signal])

        #results.append([ticker, entry_signal])
        time.sleep(2)  # Add a delay of 1 second between requests

    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["Asset", "Entry_Signal"])
    return df_results



In [ ]:
# Multi-time frame entry Check
#df_results = pd.DataFrame(results).dropna()
etfs_to_check = df_o['Asset'].tolist()
# for quick testing
#etfs_to_check  =['EZA', 'GM', 'MU', 'LRCX', 'NVDA', 'CAT', 'WDC','ILF']
df_signals = check_mtf_entry(etfs_to_check)

df_signals

## Generate buy list

In [ ]:
df_final = df_signals[df_signals['Entry_Signal'] =="Entry Confirmed ✅"]
final_etfs_to_check = df_final['Asset'].tolist()

#final_etfs_to_check.remove('REMX')
buy_list = check_entry_conditions(final_etfs_to_check)

buy_list


# Find and filter correlated assets to reduce concentration risk.

In [ ]:

def get_uncorrelated_picks(tickers, ranked_picks, threshold=0.66, period="3mo", interval="1d"):
    """
    Filters a ranked list of tickers to return only uncorrelated picks.

    Parameters:
    -----------
    tickers : list
        All candidate tickers.
    ranked_picks : list
        Ranked list of tickers (best to worst).
    threshold : float
        Correlation threshold (default 0.66).
    period : str
        Data period for yfinance (default "3mo").
    interval : str
        Data interval (default "1d").

    Returns:
    --------
    final_selection : list
        List of uncorrelated tickers.
    corr_matrix : DataFrame
        Correlation matrix of daily returns.
    """
    # Step 1: Get prices
    data = yf.download(tickers, period=period, interval=interval,auto_adjust=True)["Close"]
    data = data.ffill()

    # Step 2: Convert to daily returns
    returns = data.pct_change().dropna()

    # Step 3: Correlation matrix
    corr_matrix = returns.corr()

    # Step 4: Filter uncorrelated picks
    final_selection = []
    for pick in ranked_picks:
        if all(abs(corr_matrix.loc[pick, sel]) <= threshold for sel in final_selection):
            final_selection.append(pick)

    return final_selection, corr_matrix



In [ ]:
# Apply TA filters and prioritize ETFs
results = []
buy_list = buy_list[buy_list['Entry_Signal'].isin([
    'Aline Entry',
    'True Trend Entry' ,
    'Extended Momentum Entry'
])]


for etf in buy_list['Asset'].to_list():
   df          = get_daily_data(etf)
   price       = df['Close'].iloc[-1]
   above_21EMA = price > df['21_day_EMA'].iloc[-1]
   above_50sma = price  > df['50_day_SMA'].iloc[-1]

   sma_slope_50 = df['slope50_annualized_pct'].iloc[-1]> 0
   vwap_df     = anchored_vwap(etf, lookback_weeks=2)
   vwap_sy     = anchored_vwap_old(etf, start_of_year)
   ytd_vwap    = vwap_sy['anchored_vwap'].iloc[-1]
   # MTD
   vwap_mtd     = anchored_vwap_old(etf, first_day_month)
   mtd_vwap    = vwap_mtd['anchored_vwap'].iloc[-1]
   above_mtd_vwap = price > mtd_vwap

   # WTD
   #vwap_wtd     = anchored_vwap_old(etf, first_day_week )
   #wtd_vwap    = vwap_wtd['anchored_vwap'].iloc[-1]
   print("Current price is :", price)
   print("Year to date VWAP is :", ytd_vwap)
   vwap        = vwap_df['Anchored_VWAP'].iloc[-1]
   vwap_signal = vwap_df['Signal'].iloc[-1]

   swing_high =  vwap_df['Swing_High_Price'].iloc[-1]
   above_vwap  = price > vwap
   above_ytd_vwap = price > ytd_vwap


   if  sma_slope_50 and above_mtd_vwap and vwap_signal :
    support_level, latest_price, trail,resistance = calculate_risk_reward(df)
    entry_price = latest_price + min(0.25, 0.1*trail)
    trail = 1* trail
    risk = np.abs(entry_price- support_level)
    take_profit_1=  entry_price+ (1 *risk)
    resistance_level = entry_price + (1.1 *risk)
    reward = resistance_level - entry_price
    risk_reward_ratio = reward / risk
    # Ensure risk is greater than zero before division
    if risk > 0:
        rr_ratio_1  = np.abs(take_profit_1- entry_price) / risk
        rr_ratio  = reward / risk
    else:
        rr_ratio = np.nan

    take_profit1_perc = ((take_profit_1- entry_price )/entry_price )*100
    stop_loss_perc = ((support_level- entry_price)/entry_price )*100
    take_profit_perc = ((resistance_level- entry_price )/entry_price )*100
    # Fetch the Entry_Signal from buy_list
    entry_signal = buy_list.loc[buy_list['Asset'] == etf, 'Entry_Signal'].values[0]

    # Append results with Entry_Signal
    results.append({
            "Asset": etf,
            #"Swing High": swing_high,
            "Risk-Reward": rr_ratio,
            "Stop Out Price": support_level,
            #"Take Profit1": take_profit_1,
            "Target Price": resistance_level,
            "Current Price": latest_price,
            "Entry Price": entry_price,
            "Trail Price": trail,
            "Entry Signal": entry_signal,  # Add entry signal
            "stop_loss_perc": stop_loss_perc,
            #"take_profit1_perc": take_profit1_perc,
            "take_profit_perc": take_profit_perc,
            "Anchored VWAP": vwap,
            "MTD VWAP": mtd_vwap
            #"WTD VWAP": wtd_vwap,
            #"YTD VWAP": ytd_vwap
        })

    time.sleep(2)  # Add a delay of 1 second between requests


# Sort ETFs by highest risk-to-reward ratio
try:
   df_results = pd.DataFrame(results).dropna().sort_values(by="Risk-Reward", ascending=False).reset_index(drop = True)
except Exception as e:
  print("No Asset to buy today, check back some other time!")
  df_results = pd.DataFrame({"Asset": ["No Asset available"]})

df2 = df_results.merge(df_o[['Asset','Type', 'score']], on='Asset', how='left')
df2['timestamp'] = datetime.now()
df2 = df2.sort_values(by='score', ascending=False)
df2.head()

## Sentiment Score

In [ ]:
NEWS_API_KEY = "15c99612003d4971ad86698b50ed0bd7"  # Get one free from https://newsapi.org/
LOOKBACK_DAYS = 3

# Fetch recent news
def fetch_news(ticker, lookback_days=3):
    url = f"https://newsapi.org/v2/everything?q={ticker}&language=en&from={(datetime.now() - timedelta(days=lookback_days)).date()}&apiKey={NEWS_API_KEY}"
    resp = requests.get(url).json()
    if "articles" not in resp:
        return []
    return [a["title"] for a in resp["articles"]]

# Finbert Sentiment Scoring
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def get_sentiment_scores(news_list):
    if not news_list:
        return 0
    results = finbert(news_list)
    time.sleep(2)  # Add a delay of 1 second between requests
    scores = [1 if r["label"] == "positive" else -1 if r["label"] == "negative" else 0 for r in results]
    return np.mean(scores)

def build_sentiment_table(TICKERS):
    records = []
    for ticker in TICKERS:
        print(f"Processing {ticker}...")
        news = fetch_news(ticker, LOOKBACK_DAYS)
        sentiment_score = get_sentiment_scores(news)
        combined = {
            "Ticker": ticker,
            "Sentiment": sentiment_score
        }
        records.append(combined)
    df = pd.DataFrame(records)

    # Weighted score (adjustable)
    df["Composite_Score"] = (

        df["Sentiment"].rank(pct=True)
    )

    df = df.sort_values("Composite_Score", ascending=False).reset_index(drop=True)
    return df

# Run sentiment scoring
tickers = df2['Asset'].tolist()
results = build_sentiment_table(tickers)
top_assets = results[results["Sentiment"] >= 0]

top_assets.head()


# US Stock Entries (Day Trade)

In [ ]:
# SP 500 stocks
# Fetch the Entry_Signal from buy_list
try:
  df3 = df2[df2['Asset'].isin(top_assets['Ticker'])]
  sp500_stocks_dt = df3[(df3['Type'] == 'Stock') & (df3['Entry Signal'] == 'Extended Momentum Entry')].reset_index(drop=True)
except Exception as e:
  print("No Asset to buy today, check back some other time!")
  sp500_stocks_dt = pd.DataFrame({"Asset": ["No Asset available"]})

sp500_stocks_dt




# US Stock Entries (Aline Entry)

In [ ]:
# Fetch the Entry_Signal from buy_list
# Filter US stocks for Aline Entry
try:
  us_stocks = df3[(df3['Type'] == 'Stock') & (df3['Entry Signal'].isin(['Aline Entry']))].reset_index(drop=True)

  tickers = us_stocks['Asset'].tolist()  # Replace with your list of tickers
  ranked_picks = tickers

  final_stock_selection, corr_matrix = get_uncorrelated_picks(tickers, ranked_picks, threshold=0.66)

  #print("Final uncorrelated picks:", final_selection)
  print("\nCorrelation matrix:\n", corr_matrix)

  # Keep only rows where Asset is in filtered
  filtered_stock_list = us_stocks [us_stocks ["Asset"].isin(final_stock_selection)]
except Exception as e:
  print("No Asset to buy today, check back some other time!")
  filtered_stock_list = pd.DataFrame({"Asset": ["No Asset available"]})

filtered_stock_list

# US Stock Entries (True Trend Entry)

In [ ]:
# Fetch the Entry_Signal from buy_list
# Filter US stocks for Aline Entry
try:
  us_stocks = df3[(df3['Type'] == 'Stock') & (df3['Entry Signal'].isin(['True Trend Entry']))].reset_index(drop=True)

  tickers = us_stocks['Asset'].tolist()  # Replace with your list of tickers
  ranked_picks = tickers

  final_stock_selection, corr_matrix = get_uncorrelated_picks(tickers, ranked_picks, threshold=0.66)

  #print("Final uncorrelated picks:", final_selection)
  print("\nCorrelation matrix:\n", corr_matrix)

  # Keep only rows where Asset is in filtered
  filtered_stock_list2 = us_stocks [us_stocks ["Asset"].isin(final_stock_selection)]
except Exception as e:
  print("No Asset to buy today, check back some other time!")
  filtered_stock_list2 = pd.DataFrame({"Asset": ["No Asset available"]})

filtered_stock_list2